In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
import os
import cv2

## DEFINITIONS

In [2]:
def R_fun(s,e=1,s_max=1,Rmin=0.9,Rmax=0.99):
    r=Rmax+(Rmin-Rmax)*(s/s_max)/(e-(e-1)*(s/s_max))
    return(r)

def e_log(a,e=1,a_min=1e-12,a_max=1e-5,e_min=0.9,e_max=0.99):
    a=a_max*(np.log10(a)-np.log10(a_min))/(np.log10(a_max)-np.log10(a_min))
    epsilon=e_max+(e_min-e_max)*(a/a_max)/(e-(e-1)*(a/a_max))
    return(epsilon)

In [3]:
def construction_stress_periodic_1(T,dt,d1,d2,i_min,i_max):
    # d1: stress duration
    # d2: no stress duration
    # i_min, i_max: min and max stress intensity
    
    stress=np.array([])
    
    while np.size(stress)<=int(T/dt):
        stress=np.concatenate((stress,np.zeros(int(d2/dt)),np.repeat(np.random.uniform(i_min,i_max),int(d1/dt))))
    
    return stress[:(int(T/dt)+1)]

In [4]:
def replication(B,dt,r):
    # dt: time step
    # r: replication rate

    G=np.random.binomial(B,r*dt)
    return G

def lyse_stress(B,dt,t,d,s):
    # dt: time step
    # d: death rate
    # s: resistance rate
    
    N=np.reshape(np.sum(B,1),(nb_community,-1))
    L=np.random.binomial(B,(N*d+f2(t,s))*dt)
    return L

def infection(B,dt,a):
    # dt: time step
    # a: plasmid horizontal transfer rate
    
    I=np.random.binomial(np.reshape(B[:,0],(nb_community,-1)),B[:,1:]*a*dt)
    return I

def transfert_vertical(B,e):
    # dt: time step
    # e: plasmid vertical transfer rate

    D=np.random.binomial(B,e)
    return D

def innovation(B,dt,i):
    # dt: time step
    # i: innovation rate 
    
    N=np.random.binomial(B,i*dt)
    return N

def dispersion(B,dt,g): 
    # dt: time step
    # g: dispersal rate
    
    D=np.random.binomial(B,g*dt)    
    return D

In [5]:
def draw_plot_2(B,a_res,s_res,nb_community,time,time_vector,S,xticks,yticks,color,norm=200):
    nb_mges_final=np.shape(B)[1]-1

    run=np.repeat(np.arange(1,nb_community+1),nb_mges_final)
    a_tot=np.log10(np.tile(a_res,nb_community))
    s_tot=np.tile(s_res[1:],nb_community)
    mge_1d=np.reshape(B[:,1:],-1,order='C')

    res={'run':run,'a_tot':a_tot,'s_tot':s_tot,'density_final':mge_1d/norm}
    res_df = pd.DataFrame(data=res)

    ############################## FIGURES
    fig = plt.figure(figsize=(20, 12))
    ax1 = fig.add_axes([0.05, 0.075, 0.525, 0.85]) 

    scatter=ax1.scatter(data=res_df, x="a_tot", y="s_tot", s="density_final",c="run",cmap=color,alpha=0.8)
    ax1.set_xlabel("Horizontal transfer rate ($log_{10}$)",size=20,weight="bold")
    ax1.set_ylabel("Resistance level", size=20,weight="bold")
    ax1.set_title("T="+str(time),size=30,weight="bold")

    ax1.set_xlim(-12.5,-4.5)
    ax1.set_ylim(-0.1,1.1)
    ax1.set_xticks(xticks)
    ax1.set_xticklabels(xticks,rotation=0,size=20)
    ax1.set_yticks(yticks)
    ax1.set_yticklabels(yticks,rotation=0,size=20)
    
    ax2 = fig.add_axes([0.6, 0.075, 0.3, 0.275]) 
    ax2.yaxis.tick_right()
    ax2.yaxis.set_label_position("right")
    ax2.tick_params(axis='both', labelsize=20)
    ax2.plot(time_vector,S[8],color="black")
    ax2.vlines(time,-0.1,2.1,color="tab:red",linewidth=2)
    ax2.set_ylim(-0.05,2.05)
    ax2.set_xlabel("Time",size=30,weight="bold")
    ax2.set_yticks([0,0.5,1])
    ax2.annotate("Community 9",[0.1,1.8],color="pink",size=20,weight="bold")
    ax2.annotate("Community 10",[0.1,1.6],color="palevioletred",size=20,weight="bold")
    ax2.annotate("Community 11",[0.1,1.4],color="orchid",size=20,weight="bold")
    ax2.annotate("Community 12",[0.1,1.2],color="mediumpurple",size=20,weight="bold")

    ax3 = fig.add_axes([0.6, 0.3625, 0.3, 0.275]) 
    ax3.yaxis.tick_right()
    ax3.yaxis.set_label_position("right")
    ax3.tick_params(axis='both', labelsize=20)
    ax3.plot(time_vector,S[4],color="black")
    ax3.set_ylabel("Stress intensity",size=30,weight="bold")
    ax3.vlines(time,-0.1,2.1,color="tab:red",linewidth=2)
    ax3.set_ylim(-0.05,2.05)
    ax3.set_xticklabels([])
    ax3.set_yticks([0,0.5,1])
    ax3.annotate("Community 5",[0.1,1.8],color="turquoise",size=20,weight="bold")
    ax3.annotate("Community 6",[0.1,1.6],color="steelblue",size=20,weight="bold")
    ax3.annotate("Community 7",[0.1,1.4],color="darkturquoise",size=20,weight="bold")
    ax3.annotate("Community 8",[0.1,1.2],color="blue",size=20,weight="bold")

    ax4 = fig.add_axes([0.6, 0.65, 0.3, 0.275]) 
    ax4.yaxis.tick_right()
    ax4.yaxis.set_label_position("right")
    ax4.tick_params(axis='both', labelsize=20)
    ax4.plot(time_vector,S[0],color="black")
    ax4.set_xticklabels([])
    ax4.vlines(time,-0.1,2.1,color="tab:red",linewidth=2)
    ax4.set_ylim(-0.05,2.05)
    ax4.set_yticks([0,0.5,1])
    ax4.set_title('Stress patterns',size=30,weight='bold')
    ax4.annotate("Community 1",[0.1,1.8],color="olive",size=20,weight="bold")
    ax4.annotate("Community 2",[0.1,1.6],color="yellowgreen",size=20,weight="bold")
    ax4.annotate("Community 3",[0.1,1.4],color="lightgreen",size=20,weight="bold")
    ax4.annotate("Community 4",[0.1,1.2],color="forestgreen",size=20,weight="bold")

    plt.savefig('simulated_data/unified_model_video/'+str(time)+'.png')
    plt.close()

In [6]:
def mges_incompatibles_stochastique(B0,dt,T,n,r,e,d,a,s,i,g):
    # dt: time step
    # T: final time
    # t0: initial time
    # n: number of community
    # r: vector of replication rate
    # e: vertical transfer probability
    # d: competition coefficient
    # a: vector of infection rate
    # s: vector of resistance rate
    # i: rate of mutation
    # g: diseprsion matrix
    
    nb_species_init=np.shape(B0)[1]

    for t in range(int(T/dt)):
        B0=np.int64(B0)
        B0=np.maximum(B0,0)
        B=B0
        if t%10==0:
            draw_plot_2(B,a,s,12,int(t/10),time_vector,S,xticks,yticks,color)
        # Dispersion
        for c1 in range(n):
            for c2 in range(c1+1,n):
                G1_2=dispersion(B0[c1,:],dt,g[c1,c2])
                G2_1=dispersion(B0[c2,:],dt,g[c2,c1])
                B[c1,:]=B[c1,:]+G2_1-G1_2
                B[c2,:]=B[c2,:]+G1_2-G2_1
        
        # Local processes
        R=replication(B,dt,r)
        RP=transfert_vertical(R[:,1:],e)
        RNP=R[:,1:]-RP
        L=lyse_stress(B,dt,t,d,s)
        I=infection(B,dt,a)

        B[:,1:]=B[:,1:]+RP-L[:,1:]+I
        B[:,0]=B[:,0]+R[:,0]-L[:,0]-np.sum(I,1)+np.sum(RNP,1)
        
        # Innovation
        N=innovation(B[:,1:],dt,i)
        N_sum=np.sum(N>0)
        if(N_sum>0):
            N_ind_community=np.where(N>0)[0]
            N_ind_mge=np.where(N>0)[1]
            a_new=np.random.normal(np.log10(a[N_ind_mge]),0.1,N_sum)
            a_new=np.minimum(a_new,-5)
            a_new=np.maximum(a_new,-12)
            a_new=10**a_new
            s_new=np.random.normal(s[N_ind_mge-1],0.05,N_sum)
            s_new=np.minimum(s_new,1)
            s_new=np.maximum(s_new,0)
            a=np.append(a,a_new)
            s=np.append(s,s_new)
            r_new=R_fun(s_new,1)
            e_new=e_log(a_new,1)
            r=np.append(r,r_new)
            e=np.append(e,e_new)
            B[:,1:]=B[:,1:]-N
            B_new=np.zeros((nb_community,N_sum))
            N_ind=np.where(N>0)
            for j in range(N_sum):
                tmp=N_ind[0][j]
                B_new[tmp,j]=1
            B=np.append(B,B_new,axis=1)
        B=np.maximum(B,0)
        # Extinction
        B_ind=np.all(B[:,1:]==0,axis=0)
        if np.sum(B_ind)>0:
            tmp=np.where(B_ind==1)[0]+1
            B=np.delete(B,tmp,axis=-1)
            r=np.delete(r,tmp)
            s=np.delete(s,tmp)
            a=np.delete(a,tmp-1)
            e=np.delete(e,tmp-1)
        B0=B
            
    return(B,a,s)

## SIMULATION

In [7]:
xticks=np.arange(-12,-4,1)
yticks=np.round(np.arange(0,1.1,0.2),1)
color=LinearSegmentedColormap.from_list("",["olive","yellowgreen","lightgreen","forestgreen","turquoise","steelblue","darkturquoise","blue","pink","palevioletred","orchid","mediumpurple"])

In [8]:
np.random.seed(42)

nb_mges=200
nb_replicats=4
nb_community=nb_replicats*3
T=1001
dt=0.1

d=1e-6
a=[]
s=[]
S=np.zeros((nb_community,int(T/dt)+1))
for i in range(nb_replicats):
    S[i,:]=construction_stress_periodic_1(T,0.1,95,5,0.9,0.9)
    S[i+nb_replicats,:]=construction_stress_periodic_1(T,0.1,5,95,0.9,0.9)
    S[i+2*nb_replicats,:]=construction_stress_periodic_1(T,0.1,50,50,0.9,0.9)

s=np.random.uniform(0.2,0.8,nb_mges)
a=10**(np.random.uniform(-11,-6,nb_mges))

time_vector=np.arange(0,T+dt,dt)

In [9]:
def f2(t,s):
    return np.reshape(S[:,t],(nb_community,-1))*(1-s)

s_bis=np.concatenate(([0],s))
r=R_fun(s,1)
r_bis=np.concatenate(([1],r))
e=e_log(a,1)


g=1e-9*np.ones((nb_community,nb_community))
B0=np.zeros((nb_community,nb_mges+1))
B0[:,0]=1000
B0[:,1:]=10
for i in range(nb_community):
    g[i,i]=0

In [12]:
if not os.path.exists('simulated_data/unified_model_video'):
    os.makedirs('simulated_data/unified_model_video')

mges_incompatibles_stochastique(B0,dt,T,12,r_bis,e,d,a,s_bis,1e-4,g)

(array([[4.40010e+04, 8.04342e+05, 4.44640e+04, ..., 0.00000e+00,
         0.00000e+00, 0.00000e+00],
        [4.56310e+04, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
         0.00000e+00, 0.00000e+00],
        [5.14340e+04, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
         0.00000e+00, 0.00000e+00],
        ...,
        [2.10930e+04, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
         0.00000e+00, 0.00000e+00],
        [9.14300e+03, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
         0.00000e+00, 0.00000e+00],
        [1.14730e+04, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
         1.00000e+00, 1.00000e+00]], shape=(12, 1499)),
 array([1.05768281e-11, 9.32785061e-12, 1.87850848e-11, ...,
        8.97021983e-06, 1.00000000e-05, 9.11796827e-06], shape=(1498,)),
 array([0.        , 1.        , 1.        , ..., 1.        , 0.93846111,
        0.23719398], shape=(1499,)))

In [13]:
image_folder = 'simulated_data/unified_model_video'
video_name = 'simulated_data/unified_model_video.avi'

# Sorted list of images
images = sorted([f for f in os.listdir(image_folder) if f.endswith('.png')],key=lambda x: int(os.path.splitext(x)[0]))

# Getting the image dimensions
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Video initialization
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'XVID'), 24, (width, height))

# Adding each image
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    video.write(frame)

# Closing
video.release()
cv2.destroyAllWindows()